# Volatility Scaling & Portfolio Analysis

This notebook demonstrates how to:
1. Load and validate data.
2. Handle missing data (short vs. long gaps).
3. Adjust returns to a target volatility in-sample, then apply the same scaling out-of-sample.
4. Compute Sharpe, Sortino, Max Drawdown.
5. Provide multiple fund selection modes (all, random sample, manual).
6. Calculate portfolio results (equal-weight and custom-weight).
7. Output in-sample and out-of-sample results to Excel with formatting.

**Note**: The manual fund selection and custom weights features are partially implemented. In a real interactive workflow, you would wire widget selections and weights into the final analysis.

In [1]:
# ============ 1. SETUP CELL ============

import logging
import sys
import numpy as np
import pandas as pd
import os
import math
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox
from IPython.display import display, clear_output
from IPython.utils.capture import capture_output
from ipyfilechooser import FileChooser
import datetime
import random
import warnings

# If you need to install these packages on your environment, uncomment:
!{sys.executable} -m pip install --quiet ipywidgets openpyxl xlsxwriter

# For exporting to Excel with styling
import xlsxwriter

# Set up logging to console
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format="%(levelname)s: %(message)s"
)

logging.info("Logging started. Volatility Scaling & Portfolio Analysis Notebook initialized.")

# (Optional) If widgets aren't enabled, run:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix

print("Setup complete.")

INFO: Logging started. Volatility Scaling & Portfolio Analysis Notebook initialized.
Setup complete.


## 2. Data Loading
Here we create options to load a dataset from a local file or a GitHub repository.

In [2]:
def identify_risk_free_fund(df):
    """
    Identify which column (after 'Date') is the risk-free rate by smallest stdev among columns.
    """
    numeric_cols = df.columns[1:]  # skip the Date column
    stdevs = {}
    for col in numeric_cols:
        vals = df[col].dropna()
        if len(vals) > 0:
            stdevs[col] = vals.std()
        else:
            stdevs[col] = np.inf

    rf_col = min(stdevs, key=stdevs.get)
    logging.info(f"Identified '{rf_col}' as the risk-free column (lowest stdev).")
    return rf_col


# ------------------------------------------------------------------------------
# 1) Helper to read a local CSV robustly (handles BOMs and minor parsing issues)
# ------------------------------------------------------------------------------
def robust_read_csv(path):
    """
    Try loading `path` as CSV in three ways:
    1. Default C engine
    2. BOM-stripped with the Python engine
    3. Skip bad lines with the Python engine
    """
    try:
        return pd.read_csv(path)
    except Exception as e1:
        print("Default read_csv failed:", e1)

    try:
        return pd.read_csv(path, sep=",", encoding="utf-8-sig", engine="python")
    except Exception as e2:
        print("utf-8-sig + python engine failed:", e2)

    return pd.read_csv(
        path,
        sep=",",
        engine="python",
        encoding="utf-8-sig",
        skip_blank_lines=True,
        on_bad_lines="skip",    # for pandas ≥ 1.3
    )

# ------------------------------------------------------------------------------
# 2) Build the widgets
# ------------------------------------------------------------------------------
source_info = widgets.HTML(
    "<b>Step 1: Choose your CSV</b><br>"
    "<i>Remember:</i> If you included any index columns (e.g. S&P 500, MSCI World, SG Trend), they "
    "must appear to the right of all fund columns in your sheet."
)


source_dropdown = widgets.Dropdown(
    options=['Local', 'GitHub'],
    value='Local',
    description='Data Source:',
    style={'description_width': '120px'}
)

# FileChooser for Local mode
fc = FileChooser(os.getcwd())
fc.title = "<b>Select local CSV file</b>"

# Text box for GitHub raw URL
github_text = widgets.Text(
    value=(
        "https://raw.githubusercontent.com/stranske/Trend_Model_Project/"
        "main/data/TrendData.csv"
    ),
    description="GitHub URL:",
    layout=widgets.Layout(width="80%"),
)

# Ask how many index columns are on the far right
n_indices_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=10,  # adjust if you expect more than 10 indices
    description='# Index cols:',
    style={'description_width': '120px'},
    tooltip="Enter the number of index columns at the far right of your CSV"
)

# Load button
load_button = widgets.Button(description="Load Data", button_style="success")

# Output area for status and debug prints
output_area = widgets.Output()

# ------------------------------------------------------------------------------
# 3) Show/hide widgets depending on source selection
# ------------------------------------------------------------------------------
def on_source_change(change):
    if change["new"] == "Local":
        fc.layout.display = "block"
        github_text.layout.display = "none"
    else:
        fc.layout.display = "none"
        github_text.layout.display = "block"

# Initially, GitHub textbox is hidden; FileChooser is visible
github_text.layout.display = "none"
fc.layout.display = "block"

source_dropdown.observe(on_source_change, names="value")

ui_load = widgets.VBox([
    source_info,
    source_dropdown,
    fc,
    github_text,
    n_indices_widget,   # ← new widget here
    load_button,
    output_area
])

# ------------------------------------------------------------------------------
# 4) Callback for the Load button (auto-detect date format)
# ------------------------------------------------------------------------------
def on_load_clicked(_):
    global df, fund_cols, indices_list, rf_col            # declare df as global
    with output_area:
        output_area.clear_output()
        src_choice = source_dropdown.value
        n_indices = int(n_indices_widget.value)

        # 4a) Load the DataFrame
        try:
            if src_choice == 'Local':
                local_path = fc.selected  # FileChooser’s selected path
                if not local_path or not os.path.exists(local_path):
                    print(f"Error: Local file not found:\n  {local_path}")
                    return
                print(f"Loading from local file:\n  {local_path}")
                df = pd.read_csv(local_path)
            else:
                github_url = github_text.value.strip()
                if not github_url:
                    print("Error: Please enter a valid GitHub raw URL.")
                    return
                print(f"Loading from GitHub URL:\n  {github_url}")
                df = pd.read_csv(github_url)
        except Exception as e:
            print("Failed to load CSV:", e)
            return

        # 4b) Debug: print columns & a few rows
        print("Columns found in DataFrame:", df.columns.tolist())
        display(df.head(3))

        # 4c) Identify which column is the date
        date_col = None
        for candidate in ["Date", "DATE", "date"]:
            if candidate in df.columns:
                date_col = candidate
                break

        if date_col is None:
            print("Error: No column named 'Date' / 'DATE' / 'date' found.")
            print("Please check the column names above and adjust code accordingly.")
            return

        date_col = 'Date'

        # 1) Show the first few raw date strings (un‐parsed) so we can inspect them
        raw_samples = df[date_col].dropna().astype(str).head(10).tolist()
        print(f"Raw {date_col} samples (first 10 non‐null): {raw_samples!r}")

        # 2) Strip leading/trailing whitespace from every entry
        df[date_col] = df[date_col].astype(str).str.strip()

        # 3) Now attempt strict "%m/%d/%Y" parsing
        parsed = pd.to_datetime(df[date_col], format="%m/%d/%Y", errors="coerce", infer_datetime_format=True)
        num_valid = parsed.notna().sum()
        print(f"Number of rows matching '%m/%d/%Y' exactly: {num_valid} / {len(df)}")

        if num_valid > 0:
            df[date_col] = parsed
            print(f"Parsing with '%m/%d/%Y' succeeded for {num_valid} rows.")
        else:
            print(
                "Warning: No rows matched '%m/%d/%Y'. "
                "Falling back to generic pd.to_datetime(...)."
            )
            df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

        # 5) Drop any rows where parsing still failed
        before_drop = len(df)
        df.dropna(subset=[date_col], inplace=True)
        dropped = before_drop - len(df)
        if dropped:
            print(f"Dropped {dropped} rows where '{date_col}' could not be parsed.")
        
        # 6) Sort and reset index
        df.sort_values(by=date_col, inplace=True)
        df.reset_index(drop=True, inplace=True)
        
        # 7) Show the final parsed dates
        print(f"After parsing, first 3 {date_col} values:")
        print(df[[date_col]].head(3))
        print(f"Loaded {len(df)} rows successfully.")


        # 4d) Identify risk-free column
        try:
            rf_col = identify_risk_free_fund(df)
            print(f"Identified risk-free column as: '{rf_col}'")
        except Exception as e:
            print("Error identifying risk-free column:", e)
            return

        # 4f) Determine fund_cols vs indices_list based on n_indices
        all_cols = df.columns.tolist()
        # We assume “Date” and rf_col are present. Everything else is a candidate.
        if 'Date' not in all_cols or rf_col not in all_cols:
            print("Error: 'Date' or risk-free column not found in DataFrame columns.")
            return
        
        # Build list of all columns except 'Date' and rf_col
        remaining = [c for c in all_cols if c not in ['Date', rf_col]]
        
        if n_indices > len(remaining):
            print(
                f"Error: You asked for {n_indices} index columns, but only "
                f"{len(remaining)} columns remain after 'Date' and '{rf_col}'."
            )
            return

        if n_indices > 0:
            indices_list = remaining[-n_indices:]
            fund_cols    = remaining[:-n_indices]
        else:
            indices_list = []
            fund_cols    = remaining[:]
        print("\n>> Debug (post‐load): fund_cols =", fund_cols)
        print(">> Debug (post‐load): indices_list =", indices_list, "\n")
        
        # 2F) Print out what we found
        print(f"Detected fund columns ({len(fund_cols)}): {fund_cols}")
        print(f"Detected index columns ({len(indices_list)}): {indices_list}")
        print(
            "Data loaded and classified successfully.\n"
            "Proceed to Step 2 (Run Analysis)."
        )

 
        # 8) Confirm that df is now in global scope
        print(">> df defined with", len(df), "rows and columns:", df.columns.tolist())

# 5) Wire up and display the UI
load_button.on_click(on_load_clicked)


display(ui_load)


## 3. Utility Functions
Here we define date parsing, consecutive gap checks, data filling, risk-free identification, return calculations, etc.

In [17]:

def consecutive_gaps(series, threshold=3):
    """
    Check if a series (sorted chronologically) has >= threshold consecutive NaNs.
    Return True if such a gap exists, False otherwise.
    """
    consecutive = 0
    for val in series:
        if pd.isna(val):
            consecutive += 1
        else:
            consecutive = 0
        if consecutive >= threshold:
            return True
    return False

def fill_short_gaps_with_zero(series):
    """
    Given a pandas Series indexed by Date (month‐ends), wherever there is
    a run of 1 or 2 consecutive NaNs, replace them with 0.0. If a run of 3
    or more NaNs appears, leave those NaNs intact.
    """
    isnan = series.isna().astype(int)
    run_lengths = isnan.groupby((isnan == 0).cumsum()).transform('sum')
    filled = series.copy()
    mask_short = (isnan == 1) & (run_lengths <= 2)
    filled[mask_short] = 0.0
    return filled

def annualize_return(monthly_returns):
    """
    Annualized (geometric) return from monthly returns in decimal form.
    """
    valid_rets = monthly_returns.dropna()
    if len(valid_rets) == 0:
        return np.nan
    growth_factor = (1 + valid_rets).prod()
    n_months = len(valid_rets)
    if growth_factor <= 0:
        return -1.0
    ann_ret = growth_factor**(12.0 / n_months) - 1
    return ann_ret

def annualize_volatility(monthly_returns):
    """
    Annualized stdev of monthly returns, i.e. stdev * sqrt(12).
    """
    valid_rets = monthly_returns.dropna()
    if len(valid_rets) < 2:
        return np.nan
    return valid_rets.std() * np.sqrt(12)

def sharpe_ratio(monthly_returns, rf_series):
    """
    Annualized Sharpe ratio = (annual_excess_return) / (annual_excess_vol).
    """
    df = pd.DataFrame({'r': monthly_returns, 'rf': rf_series}).dropna()
    if len(df) < 2:
        return np.nan
    excess = df['r'] - df['rf']
    growth_factor = (1 + excess).prod()
    n_months = len(excess)
    if growth_factor <= 0:
        return np.nan
    ann_excess_ret = growth_factor**(12.0 / n_months) - 1
    ann_excess_vol = excess.std() * np.sqrt(12)
    if ann_excess_vol == 0:
        return np.nan
    return ann_excess_ret / ann_excess_vol

def sortino_ratio(monthly_returns, rf_series):
    """
    Annualized Sortino ratio = (annual_excess_return) / (annual_downside_stdev).
    """
    df = pd.DataFrame({'r': monthly_returns, 'rf': rf_series}).dropna()
    if len(df) < 2:
        return np.nan
    excess = df['r'] - df['rf']

    growth_factor = (1 + excess).prod()
    n_months = len(excess)
    if growth_factor <= 0:
        return np.nan
    ann_excess_ret = growth_factor**(12.0 / n_months) - 1

    negative_mask = excess < 0
    negative_returns = excess[negative_mask]
    if len(negative_returns) == 0:
        return np.inf  # no negative => infinite sortino
    downside_stdev = negative_returns.std() * np.sqrt(12)
    return ann_excess_ret / downside_stdev

def max_drawdown(monthly_returns):
    """
    Compute max drawdown from monthly returns in decimal form.
    """
    valid_rets = monthly_returns.dropna()
    if len(valid_rets) == 0:
        return np.nan
    wealth_index = (1 + valid_rets).cumprod()
    rolling_max = wealth_index.cummax()
    dd_series = 1 - (wealth_index / rolling_max)
    return dd_series.max()

def calc_portfolio_returns(weights, df_returns):
    """
    Compute monthly portfolio returns (Series) as weighted sum of columns in df_returns.
    """
    return (df_returns * weights).sum(axis=1)

print("Utility functions loaded.")


Utility functions loaded.


## 4. Widgets & User Inputs
Here we define some IPython widgets for in-sample/out-of-sample dates, target volatility, monthly cost, etc.

In [18]:
# ─────────────── Widget Setup + Callback ───────────────

# ─── Phase 1: Date / Vol / Cost / Run Button ───
# ─── Phase 1: Date / Vol / Cost / Run Button ───

in_sample_start     = widgets.Text(value='2005-07', description='In-Sample Start:')
in_sample_end       = widgets.Text(value='2008-06', description='In-Sample End:')
out_sample_start    = widgets.Text(value='2008-07', description='Out-Sample Start:')
out_sample_end      = widgets.Text(value='2009-06', description='Out-Sample End:')

target_vol_widget   = widgets.FloatText(value=0.25, description='Target Vol:')
monthly_cost_widget = widgets.FloatText(value=0.0033, description='Monthly Cost:')

run_button = widgets.Button(description='Run Analysis', button_style='success')

# ← Replace any previous “output_area = widgets.Output()” with the block below:
output_area = widgets.Output(
    layout={
        'border': '1px solid gray',
        'height': '300px',
        'overflow_y': 'auto'   # optional: let it scroll if there’s too much text
    }
)

# Pack only these into main_inputs for now—no display() yet.
main_inputs = widgets.VBox([
    in_sample_start,
    in_sample_end,
    out_sample_start,
    out_sample_end,
    target_vol_widget,
    monthly_cost_widget,
    run_button
])

# ─── Phase 2: Build & Hide Mode / Random-Size / “Fund Table” placeholders ───

# A) Mode dropdown (hidden initially)
selection_mode_widget = widgets.Dropdown(
    options=[('All Funds','all'), ('Random Sample','random'), ('Manual','manual')],
    value='all',
    description='Mode:'
)
selection_mode_widget.layout.display = 'none'

# B) Random Sample Size field (hidden initially)
random_sample_size_widget = widgets.IntText(
    value=5, description='Sample Size:'
)
random_sample_size_widget.layout.display = 'none'

# C) Placeholder containers for the Manual‐funds checkboxes & weights
fund_table_container = widgets.VBox([])   # we will populate it later
fund_table_container.layout.display = 'none'

# D) Whenever Mode changes, show/hide “Sample Size” vs. “Fund Table”
def _on_mode_change(change):
    new_mode = change['new']
    if new_mode == 'random':
        random_sample_size_widget.layout.display = 'block'
        fund_table_container.layout.display   = 'none'
    elif new_mode == 'manual':
        random_sample_size_widget.layout.display = 'none'
        fund_table_container.layout.display   = 'block'
    else:  # 'all'
        random_sample_size_widget.layout.display = 'none'
        fund_table_container.layout.display   = 'none'

selection_mode_widget.observe(_on_mode_change, names='value')

# ─── Phase 3: Wire up “Run Analysis” + final display(...) ───

phase_counter = {"step": 1}

def on_run_clicked(b):
    with output_area:
        clear_output()

        # 1) Read dates/vol/cost
        in_start_str     = in_sample_start.value.strip()
        in_end_str       = in_sample_end.value.strip()
        out_start_str    = out_sample_start.value.strip()
        out_end_str      = out_sample_end.value.strip()
        target_vol_val   = target_vol_widget.value
        monthly_cost_val = monthly_cost_widget.value

        # 2) First click = reveal Mode dropdown + hide everything else
        if phase_counter["step"] == 1:
            # Quick check: ensure date strings parse
            try:
                pd.to_datetime(in_start_str, format="%Y-%m")
                pd.to_datetime(in_end_str,   format="%Y-%m")
                pd.to_datetime(out_start_str, format="%Y-%m")
                pd.to_datetime(out_end_str,   format="%Y-%m")
            except Exception:
                print("❌ Invalid date formats. Please use YYYY-MM and click Run again.")
                return

            print("▶ Dates accepted. Now pick Mode (All / Random / Manual). " 
                  "If Manual, you’ll then see the valid funds appear.")
            selection_mode_widget.layout.display = 'block'
            phase_counter["step"] = 2
            return

        # 3) After first click, Mode is visible. If Mode changed to Manual, we need to
        #    build the fund_table with *only* those funds that pass the full‐history + no‐3-NA rule.
        if phase_counter["step"] == 2:
            mode_val  = selection_mode_widget.value
            rnd_n_val = random_sample_size_widget.value if mode_val == 'random' else None

            # If Manual mode is chosen, build fund_checkboxes & weight_boxes now:
            if mode_val == 'manual':
                # 3a) Determine which funds are “valid” for this date range.
                #     We reuse your existing `select_funds(...)` function, forcing selection_mode='all'.
                try:
                    in_sdate  = pd.to_datetime(in_start_str, format="%Y-%m")
                    in_edate  = pd.to_datetime(in_end_str,   format="%Y-%m") + pd.offsets.MonthEnd(0)
                    out_sdate = pd.to_datetime(out_start_str, format="%Y-%m")
                    out_edate = pd.to_datetime(out_end_str,   format="%Y-%m") + pd.offsets.MonthEnd(0)
                except Exception as e:
                    print("❌ Error parsing dates. Please re‐enter and click Run again.")
                    return

                # Build the “candidate fund_columns” list (exclude Date/rf/index columns)
                all_cols = [c for c in df.columns if c not in ['Date', identify_risk_free_fund(df)] + indices_list]
                # Call select_funds with selection_mode='all' to get the full list of valid funds.
                valid_funds = select_funds(
                    df,
                    identify_risk_free_fund(df),
                    fund_columns=all_cols,
                    in_sdate=in_sdate,
                    in_edate=in_edate,
                    out_sdate=out_sdate,
                    out_edate=out_edate,
                    selection_mode='all',
                    random_n=None
                )

                if not valid_funds:
                    print("❌ No funds satisfy the full‐history & gap rules for these dates.")
                    return

                # 3b) Build a fresh table of checkboxes + IntText’s for valid_funds
                fund_table_rows = []
                fund_checkboxes = []
                weight_boxes    = []

                for f in valid_funds:
                    cb = widgets.Checkbox(
                        value=False,
                        description=f,
                        layout=widgets.Layout(width='200px')
                    )
                    tb = widgets.BoundedIntText(
                        value=0,
                        min=0,
                        max=100,
                        step=1,
                        description="%",          # user will type an integer percent
                        layout=widgets.Layout(width='160px')
                    )
                    tb.disabled = True  # only enabled when its checkbox is clicked
                    
                    fund_checkboxes.append(cb)
                    weight_boxes.append(tb)

                    row = widgets.HBox([
                        cb,
                        weight_boxes[-1]
                    ], layout=widgets.Layout(margin="2px 0px"))
                    fund_table_rows.append(row)

                # Replace whatever was in fund_table_container with this new VBox
                fund_table_container.children = [widgets.VBox(fund_table_rows)]
                fund_table_container.layout.display = 'block'  # reveal it

                # 3c) Wire up each checkbox so its weight box toggles on/off
                def _on_cb_change(change):
                    # ‘change.owner’ is the Checkbox that toggled
                    idx = fund_checkboxes.index(change.owner)
                    weight_boxes[idx].disabled = not change.new

                for cb in fund_checkboxes:
                    cb.observe(_on_cb_change, names='value')

            # Finally, show the random_sample_size_widget if needed
            if mode_val == 'random':
                random_sample_size_widget.layout.display = 'block'

            print("▶ Now the Mode/sampling UI is visible. If Manual, pick funds + enter weights. Then click Run again.")
            phase_counter["step"] = 3
            return

        # 4) After second click, everything (Mode + any Manual selections or random size) is set,
        #    so we can finally run `run_analysis(...)` for real:
        mode_val  = selection_mode_widget.value
        rnd_n_val = random_sample_size_widget.value if mode_val == 'random' else None

        custom_weights = None
        if mode_val == 'manual':
            # Gather which checkboxes were checked, and read their weight_boxes
            selected = []
            weight_dict = {}
            # The fund_checkboxes + weight_boxes only exist if step>=2 and mode=='manual'.
            # We can grab them from fund_table_container.children.
            table_vbox = fund_table_container.children[0]   # the VBox we built above
            for row_hbox in table_vbox.children:
                cb, tb = row_hbox.children
                if cb.value:
                    selected.append(cb.description)
                    weight_dict[cb.description] = tb.value

            total_w = sum(weight_dict.values())
            if not selected:
                print("⚠️ No funds checked—pick at least one fund.")
                return
            if total_w != 100:
                print(f"⚠️ Weights sum to {total_w}, but must be exactly 100. Fix and Run again.")
                return

            custom_weights = weight_dict

        # 5) Now run the analysis for real
        print("▶ Running analysis with:")
        print(f"   In:   {in_start_str} → {in_end_str}")
        print(f"   Out:  {out_start_str} → {out_end_str}")
        print(f"   Vol:  {target_vol_val:.3%}, Cost: {monthly_cost_val:.3%}")
        print(f"   Mode: {mode_val}" + (f", Random N={rnd_n_val}" if mode_val=='random' else ""))
        if mode_val == 'manual':
            print(f"   Manual funds = {list(custom_weights.keys())}")
            print(f"   Weights = {custom_weights}")

        try:
            results = run_analysis(
                df,
                in_start=in_start_str,
                in_end=in_end_str,
                out_start=out_start_str,
                out_end=out_end_str,
                target_vol=target_vol_val,
                monthly_cost=monthly_cost_val,
                selection_mode=mode_val,
                random_n=rnd_n_val,
                custom_weights=custom_weights
            )
        except Exception as e:
            print("❌ Error inside run_analysis():", e)
            return

        if results is None or not results.get("selected_funds"):
            print("❌ No valid funds remain after filtering.")
            return

        # 6) Print a brief summary & export to Excel
        sf = results["selected_funds"]
        print("✅ Analysis complete:")
        print(f"   Funds selected: {len(sf)} → {sf}")
        print("   Manual funds & weights:")
        if custom_weights is not None:
            print("   Manual funds & weights:")
            for f, w in custom_weights.items():
                print(f"     • {f}: {w}%")
        if "in_ew_stats" in results:
            ir, iv, isr, *_ = results["in_ew_stats"]
            print(f"   In‐Sample EW → R {ir*100:.2f}%, V {iv*100:.2f}%, S {isr:.2f}")
        if "out_ew_stats" in results:
            or_, ov, osr, *_ = results["out_ew_stats"]
            print(f"   Out‐Sample EW → R {or_*100:.2f}%, V {ov*100:.2f}%, S {osr:.2f}")

        print(">> about to write Excel with custom_weights =", custom_weights)

        # Add the filename
        global fname
        fname = f"IS_{in_start_str}_{out_start_str}.xlsx"
    
        export_to_excel(
            results,
            df,            # your full DataFrame
            fname,         
            in_start_str,  
            in_end_str,    
            out_start_str, 
            out_end_str    
        )
        print(f"   Excel file created: IS_{in_start_str}_{out_start_str}.xlsx")
        import inspect
        print(inspect.getsource(export_to_excel))


run_button.on_click(on_run_clicked)

# ─── Finally, show everything (Phase 1 + Phase 2 placeholders + output area) ───

master_vbox = widgets.VBox([
    main_inputs,
    selection_mode_widget,
    random_sample_size_widget,
    fund_table_container
])



## 5. Fund Selection
Filters out columns that represent the risk-free rate or contain "index" in the name, then handles the selection mode (all, random, or manual).

In [19]:
def select_funds(
    df,                 # full DataFrame with datetime64 ‘Date’
    rf_col,             # name of risk‐free column
    fund_columns,       # list of candidate funds (should be your 27)
    in_sdate, in_edate, # pd.Timestamps for in‐sample window
    out_sdate, out_edate,# pd.Timestamps for out‐sample window
    selection_mode='all',
    random_n=8
):
    """
    1) Start from fund_columns.  
    2) Filter out any that contain 'index' in name.  
    3) Keep only those with no NaN anywhere in both windows.  
    4) Keep only those with < 3 consecutive NaNs in each window.  
    5) Return based on selection_mode.
    """
    # Step 1: base list
    candidates = fund_columns.copy()
    print(f"DEBUG [select_funds]: initial candidates (n={len(candidates)}): {candidates}")

    # Step 2: drop any with “index” in name (case‐insensitive)
    no_index = [f for f in candidates if 'index' not in f.lower()]
    print(f"DEBUG [select_funds]: after dropping 'index' (n={len(no_index)}): {no_index}")

    # Step 3: full‐history check (no NaN at all in each window)
    full_hist = []
    for f in no_index:
        in_sub  = df[(df['Date'] >= in_sdate)  & (df['Date'] <= in_edate)][f]
        out_sub = df[(df['Date'] >= out_sdate) & (df['Date'] <= out_edate)][f]
        if in_sub.notna().all() and out_sub.notna().all():
            full_hist.append(f)
    print(f"DEBUG [select_funds]: after full‐history check (n={len(full_hist)}): {full_hist}")

    # Step 4: no 3‐consecutive‐NaNs check
    after_run_check = []
    for f in full_hist:
        # In‐sample gap runs
        sub_in   = df[(df['Date'] >= in_sdate) & (df['Date'] <= in_edate)][f]
        isnan_in = sub_in.isna().astype(int)
        run_len_in = isnan_in.groupby((isnan_in == 0).cumsum()).sum()
        max_run_in = run_len_in.max() if not run_len_in.empty else 0

        # Out‐sample gap runs
        sub_out   = df[(df['Date'] >= out_sdate) & (df['Date'] <= out_edate)][f]
        isnan_out = sub_out.isna().astype(int)
        run_len_out = isnan_out.groupby((isnan_out == 0).cumsum()).sum()
        max_run_out = run_len_out.max() if not run_len_out.empty else 0

        # Keep only if both max runs < 3
        if max_run_in < 3 and max_run_out < 3:
            after_run_check.append(f)

    print(f"DEBUG [select_funds]: after run‐length check (n={len(after_run_check)}): {after_run_check}")

    # Step 5: selection_mode
    if selection_mode == 'all':
        return after_run_check

    if selection_mode == 'random':
        if len(after_run_check) <= random_n:
            warnings.warn(
                f"Fewer valid funds ({len(after_run_check)}) than sample size ({random_n}). Returning all."
            )
            return after_run_check
        return random.sample(after_run_check, random_n)

    # Manual fund selection
    
    return after_run_check

print("select_funds (replaced) is ready.")


select_funds (replaced) is ready.


## 6. Custom Weights
Displays an integer text widget for each fund, requiring the sum of weights to be 100.

In [20]:
def get_custom_weights(selected_funds):
    """
    Display widgets for each fund to enter weights. Validate sum=100.
    Returns dict {fund: weight_decimal}.
    """
    weight_widgets = {}
    for fund in selected_funds:
        w = widgets.BoundedIntText(
            value=0,
            min=0,
            max=100,
            description=f"{fund}",
            layout=widgets.Layout(width='250px')
        )
        weight_widgets[fund] = w
    
    confirm_button = widgets.Button(
        description='Confirm Weights',
        button_style='success'
    )
    error_label = widgets.Label(value='', layout=widgets.Layout(color='red'))
    
    box = VBox(list(weight_widgets.values()) + [confirm_button, error_label])
    display(box)
    
    weights_container = {}
    
    def on_confirm_clicked(_):
        total = sum(w.value for w in weight_widgets.values())
        if total != 100:
            error_label.value = f"Error: Weights sum to {total}, must be 100."
            weights_container.clear()
        else:
            for fund, wdg in weight_widgets.items():
                weights_container[fund] = wdg.value / 100.0
            error_label.value = "Weights confirmed!"
    
    confirm_button.on_click(on_confirm_clicked)
    return weights_container

print("get_custom_weights function ready.")

get_custom_weights function ready.


## 7. Analysis (In-Sample & Out-of-Sample)
The `run_analysis` function orchestrates the entire process:
- Validates date inputs.
- Converts 'Date' column.
- Identifies risk-free column.
- Fills short gaps.
- Selects funds.
- Computes in-sample scaling factors and applies them in- and out-of-sample.
- Computes individual fund stats and portfolio stats.

In [21]:
def run_analysis(
    df,
    in_start, in_end, out_start, out_end,
    target_vol, monthly_cost,
    selection_mode='all',
    random_n=8,
    custom_weights=None    # ← new argument
):
    """
    1) Parse/validate date inputs
    2) Convert Date column if needed
    3) Identify rf_col
    4) Prepare in/out sample DataFrames
    5) CALL select_funds (with debug) and print checkpoints
    6) Compute scale_factors and wrap stats in try/except
    """

    # —(1) Parse input dates (YYYY-MM) into actual month-end timestamps
    in_sdate  = pd.to_datetime(in_start + "-01", errors='coerce')
    in_edate  = pd.to_datetime(in_end   + "-01", errors='coerce') + pd.offsets.MonthEnd(0)
    out_sdate = pd.to_datetime(out_start + "-01", errors='coerce')
    out_edate = pd.to_datetime(out_end   + "-01", errors='coerce') + pd.offsets.MonthEnd(0)

    # checkpoint A
    print("CHECKPOINT A: Dates parsed:", in_sdate, in_edate, out_sdate, out_edate)

    # —(2) Ensure 'Date' column is datetime64
    if not np.issubdtype(df['Date'].dtype, np.datetime64):
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df.dropna(subset=['Date'], inplace=True)
        df.sort_values(by='Date', inplace=True)
        df.reset_index(drop=True, inplace=True)

    # checkpoint B
    print("CHECKPOINT B: Date column is datetime64, first dates:",
          df['Date'].iloc[0], "…", df['Date'].iloc[-1])

    # —(3) Identify risk-free column
    rf_col = identify_risk_free_fund(df)
    print(f"INFO: Identified '{rf_col}' as the risk-free column (lowest stdev).")

    # —(4) Slice into in-sample & out-sample
    in_sample_df  = df[(df['Date'] >= in_sdate)  & (df['Date'] <= in_edate)].copy()
    out_sample_df = df[(df['Date'] >= out_sdate) & (df['Date'] <= out_edate)].copy()
    in_sample_rf  = in_sample_df[rf_col]
    out_sample_rf = out_sample_df[rf_col]

    print(f"CHECKPOINT C: in_sample rows = {len(in_sample_df)}, out_sample rows = {len(out_sample_df)}")

    # —(5) Select funds (either “manual” or via select_funds)
    if selection_mode == 'manual' and custom_weights is not None:
        # In manual mode, we trust the user’s chosen tickers outright
        selected_funds = list(custom_weights.keys())
        print(f"CHECKPOINT D (manual): using custom_weights keys → {selected_funds}")
    else:
        all_fund_cols = fund_cols.copy()
        print(f"CHECKPOINT D: about to call select_funds with {len(all_fund_cols)} candidates")

        selected_funds = select_funds(
            df,
            rf_col,
            fund_columns=all_fund_cols,
            in_sdate=in_sdate,
            in_edate=in_edate,
            out_sdate=out_sdate,
            out_edate=out_edate,
            selection_mode=selection_mode,
            random_n=random_n
        )

    if len(selected_funds) == 0:
        logging.warning("No valid funds remain after filtering.")
        print("Warning: No valid funds. Try adjusting your selection or data filters.")
        return None

    print(f"CHECKPOINT E: select_funds returned {len(selected_funds)} funds → {selected_funds}")

    # indices_list must already exist as a global list of index columns
    in_sample_indices  = in_sample_df[indices_list].copy()
    out_sample_indices = out_sample_df[indices_list].copy()

    # —(6) Compute scale_factors (based on in-sample volatility)
    scale_factors = {}
    for fund in selected_funds:
        fund_in_rets = in_sample_df[fund].dropna()
        current_vol  = annualize_volatility(fund_in_rets)
        if pd.isna(current_vol) or current_vol == 0:
            scale_factors[fund] = 1.0
        else:
            scale_factors[fund] = target_vol / current_vol

    print("CHECKPOINT F: scale_factors computed (showing first 5):",
          {f: scale_factors[f] for f in selected_funds[:5]})

    # Pre-allocate DataFrames for scaled returns
    in_sample_scaled  = pd.DataFrame(index=in_sample_df.index, columns=selected_funds)
    out_sample_scaled = pd.DataFrame(index=out_sample_df.index, columns=selected_funds)

    # ── Wrap everything from “scaling + stats” in a try/except ──
    try:
        # (6a) Scale returns (with monthly cost)
        for fund in selected_funds:
            sf     = scale_factors[fund]
            adj_in = in_sample_df[fund] * sf - monthly_cost
            adj_in[adj_in < -1.0] = -1.0
            in_sample_scaled[fund] = adj_in

            if not out_sample_df.empty:
                adj_out = out_sample_df[fund] * sf - monthly_cost
                adj_out[adj_out < -1.0] = -1.0
                out_sample_scaled[fund] = adj_out

        # (6b) Helper to compute per-series stats
        def compute_stats(series, rf_series):
            r   = annualize_return(series)
            v   = annualize_volatility(series)
            sr  = sharpe_ratio(series, rf_series)
            so  = sortino_ratio(series, rf_series)
            mdd = max_drawdown(series)
            return (r, v, sr, so, mdd)

        # (6c) In-sample per-fund stats
        in_sample_stats = {}
        for fund in selected_funds:
            in_sample_stats[fund] = compute_stats(in_sample_scaled[fund], in_sample_rf)

        # (6d) Out-sample per-fund (vol-adjusted) stats
        out_sample_stats = {}
        for fund in selected_funds:
            out_sample_stats[fund] = compute_stats(out_sample_scaled[fund], out_sample_rf)

        # (6e) Out-sample per-fund (raw) stats
        out_sample_stats_raw = {}
        for fund in selected_funds:
            out_sample_stats_raw[fund] = compute_stats(out_sample_df[fund], out_sample_rf)

        # (6f) Equal-weight portfolio returns
        ew_weight_dict = { f: 1.0/len(selected_funds) for f in selected_funds }
        ew_w           = np.array([ ew_weight_dict[f] for f in selected_funds ])
        in_ew_port      = calc_portfolio_returns(ew_w,       in_sample_scaled[selected_funds])
        out_ew_port     = calc_portfolio_returns(ew_w,       out_sample_scaled[selected_funds])
        out_ew_port_raw = calc_portfolio_returns(ew_w,       out_sample_df[selected_funds])
        
        in_ew_stats      = compute_stats(in_ew_port,      in_sample_rf)
        out_ew_stats     = compute_stats(out_ew_port,     out_sample_rf)
        out_ew_stats_raw = compute_stats(out_ew_port_raw, out_sample_rf)

        # (6g) Index statistics (treat each index like “just another series”)
        in_index_stats  = {}
        out_index_stats = {}
        for idx in indices_list:
            in_index_stats[idx]  = compute_stats(in_sample_indices[idx],  in_sample_df[rf_col])
            out_index_stats[idx] = compute_stats(out_sample_indices[idx], out_sample_df[rf_col])

        # (6h) “User-weight” portfolio
        if custom_weights is None:
            # If no manual weights → just reuse the same equal‐weight array
            user_weight_dict = ew_weight_dict.copy()
            # ← **IMPORTANT FIX**: define custom_w here for the equal-weight case
            custom_w = ew_w.copy()  
        else:
            # Manual weights passed → convert percentages to decimals
            user_weight_dict = { f: custom_weights[f] / 100.0 for f in selected_funds }
            custom_w         = np.array([ user_weight_dict[f] for f in selected_funds ])

        # (6i) Compute “user-weighted” portfolio returns
        in_user_port      = calc_portfolio_returns(custom_w, in_sample_scaled[selected_funds])
        out_user_port     = calc_portfolio_returns(custom_w, out_sample_scaled[selected_funds])
        out_user_port_raw = calc_portfolio_returns(custom_w, out_sample_df[selected_funds])

        # (6j) Compute their stats
        in_user_stats      = compute_stats(in_user_port,      in_sample_rf)
        out_user_stats     = compute_stats(out_user_port,     out_sample_rf)
        out_user_stats_raw = compute_stats(out_user_port_raw, out_sample_rf)

        # (6k) Build the final results dict **(FIXED: removed duplicate 'fund_weights')**
        results = {
            'selected_funds':       selected_funds,
            'in_sample_scaled':     in_sample_scaled,
            'out_sample_scaled':    out_sample_scaled,
            'in_sample_stats':      in_sample_stats,
            'out_sample_stats':     out_sample_stats,
            'out_sample_stats_raw': out_sample_stats_raw,
            'in_ew_stats':          in_ew_stats,
            'out_ew_stats':         out_ew_stats,
            'out_ew_stats_raw':     out_ew_stats_raw,
            'in_user_stats':        in_user_stats,
            'out_user_stats':       out_user_stats,
            'out_user_stats_raw':   out_user_stats_raw,
            'ew_weights':           ew_weight_dict,     # ← now returns equal-weight dict
            'fund_weights':         user_weight_dict,   # ← returns manual weights (or equal if none)
            'indices_list':         indices_list
        }

    except Exception as e:
        print("ERROR inside stats-block:", e)
        return None

    # Everything succeeded
    return results



print("run_analysis (with checkpoints) is defined.")

run_analysis (with checkpoints) is defined.


## 8. Excel Export
Creates an Excel file with two sheets (In-Sample, Out-of-Sample) and two tables per sheet (Equal-weight and User-weight).

In [22]:
# ─── Helper #1: build the equal- and user-weight DataFrames ────────────────────────────────
def make_portfolio_dfs(results, sample='in'):
    """
    Returns (eq_df, user_df) for either 'in' or 'out' sample.
    """
    # pick the right stats:
    stats_dict      = results[f'{sample}_sample_stats']
    ew_stats        = results[f'{sample}_ew_stats']
    user_stats      = results[f'{sample}_user_stats']
    fund_weights    = results['fund_weights']
    funds           = results['selected_funds']

    def build_block(portf_label, stats, include_weights):
        rows = []
        # portfolio row
        r, v, sr, so, mdd = stats
        rows.append({
            'Fund': portf_label,
            'Weight': 100,
            'Return (%)': r,
            'Volatility (%)': v,
            'Sharpe Ratio': sr,
            'Sortino Ratio': so,
            'Max Drawdown (%)': mdd
        })
        # one‐row per fund
        for f in funds:
            r, v, sr, so, mdd = stats_dict[f]
            w = int(round(fund_weights[f] * 100)) if include_weights else None
            rows.append({
                'Fund': f,
                'Weight': w,
                'Return (%)': r,
                'Volatility (%)': v,
                'Sharpe Ratio': sr,
                'Sortino Ratio': so,
                'Max Drawdown (%)': mdd
            })
        return pd.DataFrame(rows)

    eq_df   = build_block('Equal-Weight Portfolio', ew_stats,   True)
    user_df = build_block('User-Weight Portfolio', user_stats, True)
    return eq_df, user_df


# ─── Helper #2: build an indices table ─────────────────────────────────────────────────────
def make_indices_df(df, indices, start, end):
    """
    Compute (Return, Vol, Sharpe, Sortino, MDD) for each index in [start, end],
    return a DataFrame with columns Index, Return (%), Volatility (%), …
    """
    in_s, in_e = pd.to_datetime(start+'-01'), pd.to_datetime(end+'-01') + pd.offsets.MonthEnd(0)
    rows = []
    rf = identify_risk_free_fund(df)
    sub = df.set_index('Date').loc[in_s:in_e]
    for idx in indices:
        series = sub[idx]
        rf_ser = sub[rf]
        r  = annualize_return(series)
        v  = annualize_volatility(series)
        sr = sharpe_ratio(series, rf_ser)
        so = sortino_ratio(series, rf_ser)
        mdd = max_drawdown(series)
        rows.append({
            'Index': idx,
            'Return (%)': r,
            'Volatility (%)': v,
            'Sharpe Ratio': sr,
            'Sortino Ratio': so,
            'Max Drawdown (%)': mdd
        })
    return pd.DataFrame(rows)


# ─── Helper #3: write & format one sheet ──────────────────────────────────────────────────
# Apply uniform column widths & formats
    #   (col_idx, width, format_obj)
COLUMN_SPECS = [
    (0, 28, None),
    (1, 12, 'int'),
    (2, 15, 'pct1'),
    (3, 15, 'pct1'),
    (4, 15, 'dec2'),
    (5, 15, 'dec2'),
    (6, 15, 'pct1'),
]

def build_formats(wb):
    return {
      'int':   wb.add_format({'num_format':'0'}),
      'pct1':  wb.add_format({'num_format':'0.0%'}),
      'dec2':  wb.add_format({'num_format':'0.00'}),
      'bold':  wb.add_format({'bold':True}),
    }
    
def write_portfolio_sheet(writer, sheet_name, eq_df, user_df, fmt):
    # Create & register the sheet
    ws = writer.book.add_worksheet(sheet_name)
    writer.sheets[sheet_name] = ws

    # 1) Write the two tables with keyword-only args
    eq_df.to_excel(excel_writer=writer,
                   sheet_name=sheet_name,
                   startrow=0,
                   index=False)
    user_df.to_excel(excel_writer=writer,
                     sheet_name=sheet_name,
                     startrow=len(eq_df) + 3,
                     index=False)

    for col_idx, width, fmt_key in COLUMN_SPECS:
        # if fmt_key is None, use no format; otherwise pull from fmt
        fmt_obj = fmt[fmt_key] if fmt_key else None
        ws.set_column(col_idx, col_idx, width, fmt_obj)

    # Bold the headers
    bold = fmt['bold']
    for j, h in enumerate(eq_df.columns):
        ws.write(0, j, h, fmt['bold'])
    start = len(eq_df) + 3
    for j, h in enumerate(user_df.columns):
        ws.write(start, j, h, fmt['bold'])


def write_indices_block(writer, sheet_name, df_idx, start_row, fmt):
    ws   = writer.sheets[sheet_name]
    bold = fmt['bold']

    # Title above the indices table
    ws.write(start_row - 1, 0, f"Indices ({sheet_name}):", bold)

    # Insert a blank Weight column so alignment matches portfolio sheets
    df_idx.insert(1, 'Weight', [''] * len(df_idx))

    # Write the DataFrame
    df_idx.to_excel(
        excel_writer=writer,
        sheet_name=sheet_name,
        startrow=start_row,
        index=False
    )

    # Apply the shared COLUMN_SPECS (not a local col_specs)
    for col_idx, width, fmt_key in COLUMN_SPECS:
        fmt_obj = fmt[fmt_key] if fmt_key else None
        ws.set_column(col_idx, col_idx, width, fmt_obj)

    # Bold the header row of the indices table
    for j, header in enumerate(df_idx.columns):
        ws.write(start_row, j, header, bold)

def export_to_excel(results, df, output_filename,
                    in_start, in_end, out_start, out_end):
    writer  = pd.ExcelWriter(output_filename, engine='xlsxwriter')
    wb     = writer.book
    formats = build_formats(wb)

    # 1) portfolio sheets
    in_eq,  in_user  = make_portfolio_dfs(results, sample='in')
    out_eq, out_user = make_portfolio_dfs(results, sample='out')
  
    # 2) indices sheets
    idx_in  = make_indices_df(df, results['indices_list'], in_start,  in_end)
    idx_out = make_indices_df(df, results['indices_list'], out_start, out_end)
    # compute where to start: right after user_df
    start_in  = len(in_eq)  + len(in_user)  + 6
    start_out = len(out_eq) + len(out_user) + 6

    write_portfolio_sheet(writer, f"IS {in_start}-{in_end}", in_eq,  in_user,  formats)
    write_portfolio_sheet(writer, f"OS {out_start}-{out_end}", out_eq, out_user, formats)
    write_indices_block(writer, f"IS {in_start}-{in_end}", idx_in,  start_in,  formats)
    write_indices_block(writer, f"OS {out_start}-{out_end}", idx_out, start_out, formats)

    writer.close()

    logging.info(f"Exported analysis to {output_filename} successfully.")
    print(f"Excel file created: {output_filename}")


## 8. Run Parameters

In [23]:
display(master_vbox, output_area)

Output(layout=Layout(border_bottom='1px solid gray', border_left='1px solid gray', border_right='1px solid gra…

### Using This Notebook
1. Run all cells.
2. Call `demo_run()` in a new cell to see a quick example with dummy data.
3. To use your own data, load it into a DataFrame (make sure it has a 'Date' column and decimal returns in other columns), then call `run_analysis()` and `export_to_excel()`.
4. For interactive selection, do:
   ```python
   display(ui_inputs)
   ```
   Then wire the `apply_button` to a callback function that reads the widget values and runs `run_analysis()`.
5. For custom weights, call:
   ```python
   my_weights = get_custom_weights(selected_funds)
   ```
   Then pass `my_weights` into your logic.
